In [2]:
 
import sqlite3
import pandas as pd
con = sqlite3.connect(':memory:')
cur = con.cursor()

table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')
    
    # 1 Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.
pd.read_sql(
    '''
    SELECT CustomerName, Country, Address FROM Customers
    where Country = 'Germany' or Country = 'France' or Country = 'France' or City = 'Madrid'

    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


In [2]:

# 2 Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.
pd.read_sql(
    '''
    SELECT Country, COUNT(Country)AS Count FROM Customers
    GROUP BY Country
    ORDER BY Count DESC
    LIMIT 3
    ''',
    con,
)


,Country,Count
0,USA,13
1,Germany,11
2,France,11


In [3]:
# 3 Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.
pd.read_sql(
    '''
    SELECT OrderDate, ShipperName FROM Orders
    INNER JOIN Shippers
    ON Orders.ShipperID = Shippers.ShipperID
    order by strftime(`OrderDate`, '%d.%m.%Y') asc
    LIMIT 1 OFFSET 9
    ''',
    con,
)


,OrderDate,ShipperName
0,16.07.1996,Federal Shipping


In [4]:
# 4 Выберите самый дорогой заказ, выведите список товаров с их ценами.
pd.read_sql(
    '''
    
    select ProductName, Price FROM OrderDetails
    INNER JOIN Products
    ON OrderDetails.ProductID = Products.ProductID
    where OrderID =
    (SELECT OrderID FROM OrderDetails
    INNER JOIN Products
    ON OrderDetails.ProductID = Products.ProductID
    GROUP BY OrderID
    order by SUM(Quantity*Price) desc
    LIMIT 1)

    ''',
    con,
)


,ProductName,Price
0,Sir Rodney's Marmalade,81.0
1,Côte de Blaye,263.5
2,Camembert Pierrot,34.0
3,Mozzarella di Giovanni,34.8


In [31]:
# 5 Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

pd.read_sql(
    '''
    SELECT ProductName, Quantity, OrderID FROM Products
    INNER JOIN OrderDetails
    ON OrderDetails.ProductID = Products.ProductID
    where Products.ProductID =
    (SELECT ProductID FROM OrderDetails
    GROUP BY ProductID
    order by SUM(Quantity) desc
    LIMIT 1)
    ''',
    con,
)

,ProductName,Quantity,OrderID
0,Gorgonzola Telino,20,10253
1,Gorgonzola Telino,40,10272
2,Gorgonzola Telino,15,10273
3,Gorgonzola Telino,4,10325
4,Gorgonzola Telino,25,10335
5,Gorgonzola Telino,56,10342
6,Gorgonzola Telino,30,10356
7,Gorgonzola Telino,70,10359
8,Gorgonzola Telino,20,10363
9,Gorgonzola Telino,30,10374


In [157]:
# 6  Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.
pd.read_sql(
    '''
    
    SELECT Products.SupplierID, COUNT(Products.SupplierID)AS Count, SupplierName, Country, ContactName, Phone FROM OrderDetails
    INNER JOIN Products
    ON OrderDetails.ProductID = Products.ProductID
    INNER JOIN Suppliers
    ON Suppliers.SupplierID = Products.SupplierID
    GROUP BY Products.SupplierID
    order by COUNT desc
    LIMIT 3
    ''',
    con,
) 





,SupplierID,Count,SupplierName,Country,ContactName,Phone
0,12,42,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
1,7,40,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
2,8,33,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448


In [89]:
# 7 Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.
pd.read_sql(
    '''
    select 'Brazil' as Country, SUM(OrderDetails.Quantity * Products.Price) as SUMM, CategoryName from OrderDetails
    INNER JOIN Categories
    ON Categories.CategoryID = Products.CategoryID
    INNER JOIN Products
    ON OrderDetails.ProductID = Products.ProductID
        where OrderID in
    (SELECT OrderID FROM Orders
    where CustomerID in 
    (SELECT CustomerID FROM Customers
    Where Country= 'Brazil'
    ))
    GROUP BY Products.CategoryID 
    order by SUM(OrderDetails.Quantity * Products.Price) DESC
    LIMIT 1
    ''',
    con,
)

,Country,SUMM,CategoryName
0,Brazil,13690.0,Beverages


In [158]:
# 8 Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.
pd.read_sql(
    '''
    select MAX, MIN, (MAX-MIN) as razn from
    
    (select 
    (SELECT SUM(Quantity*Price) as SUMM FROM OrderDetails
    INNER JOIN Orders
    ON OrderDetails.OrderID = Orders.OrderID
    INNER JOIN Products
    ON OrderDetails.ProductID = Products.ProductID
    where CustomerID in
    (SELECT CustomerID from Customers
    where Country = 'USA')
    GROUP BY OrderDetails.OrderID
    order by SUM(Quantity*Price) desc
    LIMIT 1) as MAX,
    (SELECT SUM(Quantity*Price) as SUMM FROM OrderDetails
    INNER JOIN Orders
    ON OrderDetails.OrderID = Orders.OrderID
    INNER JOIN Products
    ON OrderDetails.ProductID = Products.ProductID
    where CustomerID in
    (SELECT CustomerID from Customers
    where Country = 'USA')
    GROUP BY OrderDetails.OrderID
    order by SUM(Quantity*Price) asc
    LIMIT 1) as MIN) 
    ''',
    con,
)

,MAX,MIN,razn
0,7698.45,60.0,7638.45


In [171]:
# 9 Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.
pd.read_sql(
    '''
    Select COUNT(OrderID)AS Count, (LastName || ' ' || FirstName) as Name from Orders
    INNER JOIN Employees
    ON Orders.EmployeeID = Employees.EmployeeID
    where Orders.EmployeeID in (
    SELECT EmployeeID from(
    SELECT EmployeeID, (LastName || ' ' || FirstName) as Name, (substr(BirthDate,7)||substr(BirthDate,4,2)||substr(BirthDate,1,2)) as BirthDate2 FROM Employees
    order by BirthDate2 desc 
    LIMIT 3))
    GROUP BY Orders.EmployeeID
       
   
     ''',
    con,
)

,Count,Name
0,29,Davolio Nancy
1,31,Leverling Janet
2,6,Dodsworth Anne


In [175]:
# 10 Сколько банок крабового мяса всего было заказано.
pd.read_sql(
    '''
    SELECT (SUM (Quantity)*24) as SUMMCRAB FROM OrderDetails
    where ProductID = '40'
    ''',
    con,
)

,SUMMCRAB
0,6144
